### Detection of bends using corner harris

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# blur_img_cropped
dest = cv2.cornerHarris(im, 6, 3, 0.05)

In [7]:
# Results are marked through the dilated corners
dest = cv2.dilate(dest, None)

 # Normalizing
dst_norm = np.empty(dest.shape, dtype=np.float32)
cv2.normalize(dest, dst_norm, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
dst_norm_scaled = cv2.convertScaleAbs(dst_norm)

In [16]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Display blur_img_cropped in the first subplot
axes[0].imshow(im, cmap='gray')
axes[0].set_title('Image Cropped')

# Display dst_norm_scaled in the second subplot
axes[1].imshow(dst_norm_scaled, cmap='gray')
axes[1].set_title('Dst Norm Scaled')

In [15]:
_, binary_image = cv2.threshold(dst_norm_scaled, 70, 255, cv2.THRESH_BINARY)
plt.imshow(binary_image, cmap = 'gray')

In [17]:
# Find contours in the binary image
contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw contours on a copy of img_cropped
img_with_contours = img_contours.copy()
# Initialize a list to store centroids
centroids = []

# Compute centroids and draw them on the image
for contour in contours:
    # Calculate the centroid of the contour
    M = cv2.moments(contour)
    if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        centroids.append((cX, cY))

        # Draw the centroid on the image
        cv2.circle(img_with_contours, (cX, cY), 2, (255, 0, 0), -1)
plt.imshow(img_with_contours, cmap = 'gray')

### Detemining bends using Gradients

In [170]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [174]:
# reading and converting img to grayscale
image_rgb = cv2.imread('circ_cab2_upscaled.jpeg')
img = cv2.imread('circ_cab2_upscaled.jpeg', cv2.IMREAD_GRAYSCALE)

In [175]:
# Setting a threshold limit to convert all pixel below threshold to black and above to white
gray2 = np.zeros(img.shape)
gray2 [img < 210] = 255
gray2 [img > 130] = 0
plt.imshow(gray2, cmap = 'gray')

In [176]:
# output_img = output[:,pos[0]:pos[1]-1]
# for i in range(len(img_arr)):

edges = cv2.Canny(img, 100, 200)

dx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3, borderType=cv2.BORDER_REPLICATE)
dy = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3, borderType=cv2.BORDER_REPLICATE)

angle = np.zeros_like(img, dtype=np.float64)

y, x = np.where(edges > 0)
angle[y, x] = np.arctan2(dy[y, x], dx[y, x])
angle_degrees = np.degrees(angle)

plt.imshow(dy)

In [177]:
# Normalize angles to range [0, 1]
normalized_angles = (angle_degrees - angle_degrees.min()) / (angle_degrees.max() - angle_degrees.min())

# Scale to range [0, 255]
angle_visualization = (normalized_angles * 255).astype(np.uint8)
plt.imshow(angle_visualization, cmap='gray')

In [178]:
# User-defined threshold value
user_threshold_value = 100

# Binary thresholding
angle_visualization_inverted = cv2.bitwise_not(angle_visualization)
_, binary_thresholded = cv2.threshold(angle_visualization_inverted, user_threshold_value, 255, cv2.THRESH_BINARY)
# Otsu's thresholding
_, otsu_thresholded = cv2.threshold(angle_visualization_inverted, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# Plotting
plt.figure(figsize=(20, 3))

# Original visualization
plt.subplot(1, 3, 1)
plt.imshow(angle_visualization_inverted, cmap='gray')
plt.title('Original Visualization')
plt.axis('off')

# Binary thresholded image
plt.subplot(1, 3, 2)
plt.imshow(binary_thresholded, cmap='gray')
plt.title('Binary Thresholded (User-defined)')
plt.axis('off')

# Otsu's thresholded image
plt.subplot(1, 3, 3)
plt.imshow(otsu_thresholded, cmap='gray')
plt.title("Otsu's Thresholded")
plt.axis('off')

In [179]:
# img_arr = [binary_thresholded]
img_arr = (otsu_thresholded)

In [180]:
# Find contours
bends = []
for j in range(len(img_arr)):
  if j!=3:
    continue
  kernel = np.ones((5, 5), np.uint8)
  img_dilation = cv2.dilate(img_arr[j], kernel, iterations=1)
  contours, _ = cv2.findContours(img_dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Convert contours to a list so we can modify it
  contours = list(contours)

  # Iterate through contours in reverse order so we can safely remove contours from the list
  for i in reversed(range(len(contours))):
      # Calculate area of contour
      area = cv2.contourArea(contours[i])
      # If area is less than 5, remove the contour from the list
      if area < 20:
          contours.pop(i)

  # Draw remaining contours on the RGB image in red
  img_arr_rgb = cv2.cvtColor(img_arr[j], cv2.COLOR_GRAY2RGB)
  contour_image = img_arr_rgb.copy()
  cv2.drawContours(contour_image, contours, -1, (255, 0,0), -1)
  bends.append(len(contours))
# for contour in contours:
#     convexHull = cv2.convexHull(contour)
#     cv2.drawContours(contour_image, [convexHull], -1, (0, 255, 0), 1)
contour_image

In [181]:
bends

### Retrying Gradient approach

In [182]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [183]:
# reading and converting img to grayscale
image_rgb = cv2.imread('circ_cab_upscaled.jpeg')
img = cv2.imread('circ_cab_upscaled.jpeg', cv2.IMREAD_GRAYSCALE)

In [184]:
# Setting a threshold limit to convert all pixel below threshold to black and above to white
gray2 = np.zeros(img.shape)
gray2 [img < 210] = 255
gray2 [img > 130] = 0
plt.imshow(gray2, cmap = 'gray')

In [185]:
# output_img = output[:,pos[0]:pos[1]-1]
# for i in range(len(img_arr)):

edges = cv2.Canny(img, 150, 200)

dx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3, borderType=cv2.BORDER_REPLICATE)
dy = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3, borderType=cv2.BORDER_REPLICATE)

angle = np.zeros_like(img, dtype=np.float64)

y, x = np.where(edges > 0)
angle[y, x] = np.arctan2(dy[y, x], dx[y, x])
angle_degrees = np.degrees(angle)

plt.imshow(dy)

In [186]:
# Normalize angles to range [0, 1]
normalized_angles = (angle_degrees - angle_degrees.min()) / (angle_degrees.max() - angle_degrees.min())

# Scale to range [0, 255]
angle_visualization = (normalized_angles * 255).astype(np.uint8)
plt.imshow(angle_visualization, cmap='gray')

In [187]:
# User-defined threshold value
user_threshold_value = 130

# Binary thresholding
angle_visualization_inverted = cv2.bitwise_not(angle_visualization)
_, binary_thresholded = cv2.threshold(angle_visualization_inverted, user_threshold_value, 255, cv2.THRESH_BINARY)
# Otsu's thresholding
_, otsu_thresholded = cv2.threshold(angle_visualization_inverted, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# Plotting
plt.figure(figsize=(10, 6))

# Original visualization
plt.subplot(1, 3, 1)
plt.imshow(angle_visualization_inverted, cmap='gray')
plt.title('Original Visualization')
plt.axis('off')

# Binary thresholded image
plt.subplot(1, 3, 2)
plt.imshow(binary_thresholded, cmap='gray')
plt.title('Binary Thresholded (User-defined)')
plt.axis('off')

# Otsu's thresholded image
plt.subplot(1, 3, 3)
plt.imshow(otsu_thresholded, cmap='gray')
plt.title("Otsu's Thresholded")
plt.axis('off')

In [87]:
img_arr = (binary_thresholded)

In [98]:
# Find contours
bends = []
for j in range(len(img_arr)):
  if j!=3:
    continue
  kernel = np.ones((5, 5), np.uint8)
  img_dilation = cv2.dilate(img_arr[j], kernel, iterations=1)
  contours, _ = cv2.findContours(img_dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Convert contours to a list so we can modify it
  contours = list(contours)

  # Iterate through contours in reverse order so we can safely remove contours from the list
  for i in reversed(range(len(contours))):
      # Calculate area of contour
      area = cv2.contourArea(contours[i])
      # If area is less than 5, remove the contour from the list
      if area < 20:
          contours.pop(i)

  # Draw remaining contours on the RGB image in red
  img_arr_rgb = cv2.cvtColor(img_arr[j], cv2.COLOR_GRAY2RGB)
  contour_image = img_arr_rgb.copy()
  cv2.drawContours(contour_image, contours, -1, (255, 0,0), -1)
  bends.append(len(contours))
# for contour in contours:
#     convexHull = cv2.convexHull(contour)
#     cv2.drawContours(contour_image, [convexHull], -1, (0, 255, 0), 1)
contour_image


In [97]:
bends

In [108]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read and convert the image to grayscale
image_rgb = cv2.imread('circ_cab2_big.png')
img = cv2.imread('circ_cab2_big.png', cv2.IMREAD_GRAYSCALE)

# Check if the image is loaded correctly
if img is None:
    print("Error: Image not loaded. Please check the file path.")
else:
    # Threshold to convert all pixels below threshold to black and above to white
    gray2 = np.zeros(img.shape)
    gray2[img < 210] = 255
    gray2[img > 130] = 0
    plt.imshow(gray2, cmap='gray')
    plt.title('Thresholded Image')
    plt.show()

    # Edge detection using Canny
    edges = cv2.Canny(img, 100, 200)

    # Calculate gradients using Sobel operator
    dx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3, borderType=cv2.BORDER_REPLICATE)
    dy = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3, borderType=cv2.BORDER_REPLICATE)

    # Calculate gradient angle
    angle = np.zeros_like(img, dtype=np.float64)
    y, x = np.where(edges > 0)
    angle[y, x] = np.arctan2(dy[y, x], dx[y, x])
    angle_degrees = np.degrees(angle)

    # Normalize angles to range [0, 1]
    normalized_angles = (angle_degrees - angle_degrees.min()) / (angle_degrees.max() - angle_degrees.min())

    # Scale to range [0, 255]
    angle_visualization = (normalized_angles * 255).astype(np.uint8)
    plt.imshow(angle_visualization, cmap='gray')
    plt.title('Angle Visualization')
    plt.show()

    # User-defined threshold value
    user_threshold_value = 120

    # Binary thresholding
    angle_visualization_inverted = cv2.bitwise_not(angle_visualization)
    _, binary_thresholded = cv2.threshold(angle_visualization_inverted, user_threshold_value, 255, cv2.THRESH_BINARY)

    # Otsu's thresholding
    _, otsu_thresholded = cv2.threshold(angle_visualization_inverted, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Plotting
    plt.figure(figsize=(20, 3))

    # Original visualization
    plt.subplot(1, 3, 1)
    plt.imshow(angle_visualization_inverted, cmap='gray')
    plt.title('Original Visualization')
    plt.axis('off')

    # Binary thresholded image
    plt.subplot(1, 3, 2)
    plt.imshow(binary_thresholded, cmap='gray')
    plt.title('Binary Thresholded (User-defined)')
    plt.axis('off')

    # Otsu's thresholded image
    plt.subplot(1, 3, 3)
    plt.imshow(otsu_thresholded, cmap='gray')
    plt.title("Otsu's Thresholded")
    plt.axis('off')
    plt.show()

    # Use the binary thresholded image for contour detection
    img_arr = binary_thresholded

    # Kernel for dilation
    kernel = np.ones((5, 5), np.uint8)

    # Dilate the image
    img_dilation = cv2.dilate(img_arr, kernel, iterations=1)

    # Find contours
    contours, _ = cv2.findContours(img_dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Convert contours to a list so we can modify it
    contours = list(contours)

    # Iterate through contours in reverse order so we can safely remove contours from the list
    for i in reversed(range(len(contours))):
        # Calculate area of contour
        area = cv2.contourArea(contours[i])
        # If area is less than 20, remove the contour from the list
        if area < 20:
            contours.pop(i)

    # Draw remaining contours on the RGB version of the original image
    contour_image = image_rgb.copy()
    cv2.drawContours(contour_image, contours, -1, (255, 0, 0), 2)  # Draw with a thickness of 2 for visibility

    # Plot the final image with contours
    plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB))
    plt.title('Image with Contours')
    plt.axis('off')
    plt.show()

    # Print the number of bends found
    bends = len(contours)
    print(f'Number of bends found: {bends}')


In [112]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read and convert the image to grayscale
image_rgb = cv2.imread('circ_cab2_big.png')
img = cv2.imread('circ_cab2_big.png', cv2.IMREAD_GRAYSCALE)

# Check if the image is loaded correctly
if img is None:
    print("Error: Image not loaded. Please check the file path.")
else:
    # Crop the image to remove the border
    img = cv2.copyMakeBorder(img, 10, 10, 10, 10, cv2.BORDER_CONSTANT, value=255)

    # Threshold to convert all pixels below threshold to black and above to white
    gray2 = np.zeros(img.shape)
    gray2[img < 210] = 255
    gray2[img > 130] = 0
    plt.imshow(gray2, cmap='gray')
    plt.title('Thresholded Image')
    plt.show()

    # Edge detection using Canny
    edges = cv2.Canny(img, 100, 200)

    # Calculate gradients using Sobel operator
    dx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3, borderType=cv2.BORDER_REPLICATE)
    dy = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3, borderType=cv2.BORDER_REPLICATE)

    # Calculate gradient angle
    angle = np.zeros_like(img, dtype=np.float64)
    y, x = np.where(edges > 0)
    angle[y, x] = np.arctan2(dy[y, x], dx[y, x])
    angle_degrees = np.degrees(angle)

    # Normalize angles to range [0, 1]
    normalized_angles = (angle_degrees - angle_degrees.min()) / (angle_degrees.max() - angle_degrees.min())

    # Scale to range [0, 255]
    angle_visualization = (normalized_angles * 255).astype(np.uint8)
    plt.imshow(angle_visualization, cmap='gray')
    plt.title('Angle Visualization')
    plt.show()

    # User-defined threshold value
    user_threshold_value = 120

    # Binary thresholding
    angle_visualization_inverted = cv2.bitwise_not(angle_visualization)
    _, binary_thresholded = cv2.threshold(angle_visualization_inverted, user_threshold_value, 255, cv2.THRESH_BINARY)

    # Otsu's thresholding
    _, otsu_thresholded = cv2.threshold(angle_visualization_inverted, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Plotting
    plt.figure(figsize=(20, 3))

    # Original visualization
    plt.subplot(1, 3, 1)
    plt.imshow(angle_visualization_inverted, cmap='gray')
    plt.title('Original Visualization')
    plt.axis('off')

    # Binary thresholded image
    plt.subplot(1, 3, 2)
    plt.imshow(binary_thresholded, cmap='gray')
    plt.title('Binary Thresholded (User-defined)')
    plt.axis('off')

    # Otsu's thresholded image
    plt.subplot(1, 3, 3)
    plt.imshow(otsu_thresholded, cmap='gray')
    plt.title("Otsu's Thresholded")
    plt.axis('off')
    plt.show()

    # Use the binary thresholded image for contour detection
    img_arr = binary_thresholded

    # Kernel for dilation
    kernel = np.ones((5, 5), np.uint8)

    # Dilate the image
    img_dilation = cv2.dilate(img_arr, kernel, iterations=1)

    # Find contours
    contours, _ = cv2.findContours(img_dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Convert contours to a list so we can modify it
    contours = list(contours)

    # Iterate through contours in reverse order so we can safely remove contours from the list
    for i in reversed(range(len(contours))):
        # Calculate area of contour
        area = cv2.contourArea(contours[i])
        # If area is less than 20, remove the contour from the list
        if area < 20:
            contours.pop(i)

    # Draw remaining contours on the RGB version of the original image
    contour_image = image_rgb.copy()
    cv2.drawContours(contour_image, contours, -1, (255, 0, 0), 2)  # Draw with a thickness of 2 for visibility

    # Plot the final image with contours
    plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB))
    plt.title('Image with Contours')
    plt.axis('off')
    plt.show()

    # Print the number of bends found
    bends = len(contours)
    print(f'Number of bends found: {bends}')

In [113]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read and convert the image to grayscale
image_rgb = cv2.imread('circ_cab2_big.png')
img = cv2.imread('circ_cab2_big.png', cv2.IMREAD_GRAYSCALE)

# Check if the image is loaded correctly
if img is None:
    print("Error: Image not loaded. Please check the file path.")
else:
    # Crop the image to remove the border
    img = cv2.copyMakeBorder(img, 10, 10, 10, 10, cv2.BORDER_CONSTANT, value=255)

    # Adaptive thresholding to convert grayscale image to binary
    binary_thresholded = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 75, 10)

    # Plotting
    plt.figure(figsize=(20, 3))

    # Original image
    plt.subplot(1, 3, 1)
    plt.imshow(cv2.cvtColor(image_rgb, cv2.COLOR_BGR2RGB))
    plt.title('Original Image')
    plt.axis('off')

    # Binary thresholded image
    plt.subplot(1, 3, 2)
    plt.imshow(binary_thresholded, cmap='gray')
    plt.title('Adaptive Thresholded')
    plt.axis('off')

    # Dilated image
    kernel = np.ones((5, 5), np.uint8)
    img_dilation = cv2.dilate(binary_thresholded, kernel, iterations=1)
    plt.subplot(1, 3, 3)
    plt.imshow(img_dilation, cmap='gray')
    plt.title('Dilated Image')
    plt.axis('off')

    plt.show()

    # Find contours
    contours, _ = cv2.findContours(img_dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Convert contours to a list so we can modify it
    contours = list(contours)

    # Iterate through contours in reverse order so we can safely remove contours from the list
    for i in reversed(range(len(contours))):
        # Calculate area of contour
        area = cv2.contourArea(contours[i])
        # If area is less than 20, remove the contour from the list
        if area < 20:
            contours.pop(i)

    # Draw remaining contours on the RGB version of the original image
    contour_image = image_rgb.copy()
    cv2.drawContours(contour_image, contours, -1, (255, 0, 0), 2)  # Draw with a thickness of 2 for visibility

    # Plot the final image with contours
    plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB))
    plt.title('Image with Contours')
    plt.axis('off')
    plt.show()

    # Print the number of bends found
    bends = len(contours)
    print(f'Number of bends found: {bends}')

In [114]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read and convert the image to grayscale
image_rgb = cv2.imread('circ_cab2_big.png')
img = cv2.imread('circ_cab2_big.png', cv2.IMREAD_GRAYSCALE)

# Check if the image is loaded correctly
if img is None:
    print("Error: Image not loaded. Please check the file path.")
else:
    # Crop the image to remove the border
    img = cv2.copyMakeBorder(img, 10, 10, 10, 10, cv2.BORDER_CONSTANT, value=255)

    # Threshold to convert all pixels below threshold to black and above to white
    gray2 = np.zeros(img.shape)
    gray2[img < 210] = 255
    gray2[img > 130] = 0
    plt.imshow(gray2, cmap='gray')
    plt.title('Thresholded Image')
    plt.show()

    # Edge detection using Canny
    edges = cv2.Canny(img, 100, 200)

    # Calculate gradients using Sobel operator
    dx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3, borderType=cv2.BORDER_REPLICATE)
    dy = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3, borderType=cv2.BORDER_REPLICATE)

    # Calculate gradient angle
    angle = np.zeros_like(img, dtype=np.float64)
    y, x = np.where(edges > 0)
    angle[y, x] = np.arctan2(dy[y, x], dx[y, x])
    angle_degrees = np.degrees(angle)

    # Normalize angles to range [0, 1]
    normalized_angles = (angle_degrees - angle_degrees.min()) / (angle_degrees.max() - angle_degrees.min())

    # Scale to range [0, 255]
    angle_visualization = (normalized_angles * 255).astype(np.uint8)
    plt.imshow(angle_visualization, cmap='gray')
    plt.title('Angle Visualization')
    plt.show()

    # User-defined threshold value
    user_threshold_value = 120

    # Binary thresholding
    angle_visualization_inverted = cv2.bitwise_not(angle_visualization)
    _, binary_thresholded = cv2.threshold(angle_visualization_inverted, user_threshold_value, 255, cv2.THRESH_BINARY)

    # Plotting
    plt.figure(figsize=(20, 3))

    # Original visualization
    plt.subplot(1, 3, 1)
    plt.imshow(angle_visualization_inverted, cmap='gray')
    plt.title('Original Visualization')
    plt.axis('off')

    # Binary thresholded image
    plt.subplot(1, 3, 2)
    plt.imshow(binary_thresholded, cmap='gray')
    plt.title('Binary Thresholded (User-defined)')
    plt.axis('off')

    # Dilated image
    plt.subplot(1, 3, 3)
    kernel = np.ones((5, 5), np.uint8)
    img_dilation = cv2.dilate(binary_thresholded, kernel, iterations=1)
    plt.imshow(img_dilation, cmap='gray')
    plt.title('Dilated Image')
    plt.axis('off')

    plt.show()

    # Apply probabilistic Hough transform to detect lines
    lines = cv2.HoughLinesP(img_dilation, rho=1, theta=np.pi/180, threshold=50, minLineLength=30, maxLineGap=10)

    # Calculate the intersections between lines
    intersection_points = []
    for i in range(len(lines)):
        for j in range(i + 1, len(lines)):
            line1 = lines[i][0]
            line2 = lines[j][0]
            x1, y1 = line1[:2]
            x2, y2 = line1[2:]
            x3, y3 = line2[:2]
            x4, y4 = line2[2:]
            # Calculate intersection point
            det = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
            if det != 0:
                px = ((x1*y2-y1*x2)*(x3-x4)-(x1-x2)*(x3*y4-y3*x4))/det
                py = ((x1*y2-y1*x2)*(y3-y4)-(y1-y2)*(x3*y4-y3*x4))/det
                intersection_points.append((px, py))

    # Draw lines and intersection points on the RGB version of the original image
    line_image = image_rgb.copy()
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(line_image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    for point in intersection_points:
        px, py = point
        cv2.circle(line_image, (int(px), int(py)), 5, (0, 255, 0), -1)

    # Plot the final image with lines and intersection points
    plt.imshow(cv2.cvtColor(line_image, cv2.COLOR_BGR2RGB))
    plt.title('Image with Lines and Intersection Points')
    plt.axis('off')
    plt.show()

    # Print the number of bends found
    bends = len(intersection_points)
    print(f'Number of bends found: {bends}')

In [123]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read and convert the image to grayscale
image_rgb = cv2.imread('circ_cab2_upscaled.jpeg')
img = cv2.imread('circ_cab2_upscaled.jpeg', cv2.IMREAD_GRAYSCALE)

# Thresholding to isolate the cable (adjust thresholds as needed)
_, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)

# Morphological operations to enhance the bends
kernel = np.ones((5, 5), np.uint8)  # Adjust kernel size if necessary
dilated = cv2.dilate(thresh, kernel, iterations=1)
eroded = cv2.erode(dilated, kernel, iterations=1)

# Find contours
contours, hierarchy = cv2.findContours(eroded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Filter out contours based on hierarchy and area
filtered_contours = []
for i, contour in enumerate(contours):
    if hierarchy[0][i][3] == -1 and cv2.contourArea(contour) > 20:  # Child contours with sufficient area
        filtered_contours.append(contour)

# Draw remaining contours on the RGB version of the original image
contour_image = image_rgb.copy()
cv2.drawContours(contour_image, filtered_contours, -1, (255, 0, 0), 2)

# Plot the final image with contours
plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB))
plt.title('Image with Contours')
plt.axis('off')
plt.show()

print('Number of bends found: ', len(filtered_contours))

In [168]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read and convert the image to grayscale
image_rgb = cv2.imread('circ_cab2_upscaled.jpeg')
img = cv2.imread('circ_cab2_upscaled.jpeg', cv2.IMREAD_GRAYSCALE)
# Thresholding to isolate the cable (adjust thresholds as needed)
_, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)

# Morphological operations to enhance the bends
kernel = np.ones((3, 3), np.uint8)  # Smaller kernel size for finer details
dilated = cv2.dilate(thresh, kernel, iterations=2)  # More iterations to connect bends
eroded = cv2.erode(dilated, kernel, iterations=1)

plt.imshow(dilated)
# Find contours
contours, hierarchy = cv2.findContours(eroded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Filter out contours based on hierarchy and area
filtered_contours = []
for i, contour in enumerate(contours):
    # Check if contour is a child (hierarchy level 2 or 3) and has sufficient area
    if (hierarchy[0][i][3] == -1 or hierarchy[0][i][3] > 0) and 20 < cv2.contourArea(contour) < 500:
        filtered_contours.append(contour)

# Draw remaining contours on the RGB version of the original image
contour_image = image_rgb.copy()
cv2.drawContours(contour_image, filtered_contours, -1, (255, 0, 0), 2)

# Plot the final image with contours
plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB))
plt.title('Image with Contours')
plt.axis('off')
plt.show()

print('Number of bends found: ', len(filtered_contours))

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read and convert the image to grayscale
image_rgb = cv2.imread('circ_cab2_big.png')
img = cv2.imread('circ_cab2_big.png', cv2.IMREAD_GRAYSCALE)

if img is None:
    print("Error: Image not loaded. Please check the file path.")
else:
    # Apply preprocessing steps to enhance edges/bends
    # Example: Adaptive Thresholding
    img_blurred = cv2.GaussianBlur(img, (5, 5), 0)
    _, img_thresh = cv2.threshold(img_blurred, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours
    contours, hierarchy = cv2.findContours(img_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on specific criteria
    filtered_contours = []
    for contour in contours:
        # Calculate contour properties
        area = cv2.contourArea(contour)
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = float(w) / h
        if area < 0.9 * img.shape[0] * img.shape[1] and area > 50:  # Exclude large border contours and very small ones
            if aspect_ratio > 0.1 and aspect_ratio < 10:  # Filter based on aspect ratio
                filtered_contours.append(contour)

    # Draw filtered contours on the original image
    contour_image = image_rgb.copy()
    cv2.drawContours(contour_image, filtered_contours, -1, (0, 255, 0), 2)

    # Display the result
    plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB))
    plt.title('Filtered Contours')
    plt.axis('off')
    plt.show()

    # Print the number of bends found
    print(f'Number of bends found: {len(filtered_contours)}')